# Reshaping Data

## Acerca de los datos
En este notebook, utilizaremos datos diarios de temperatura de la API [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). Utilizaremos el conjunto de datos Global Historical Climatology Network - Daily (GHCND); consulte la documentación [aquí](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf).

Estos datos se recopilaron para la ciudad de Nueva York para octubre de 2018, utilizando la estación Boonton 1 (GHCND:USC00280907). Contiene:
- la temperatura mínima diaria (`TMIN`)
- la temperatura máxima diaria (`TMAX`)
- la temperatura diaria en el momento de la observación (`TOBS`)

*Nota: El NCEI forma parte de la Administración Nacional Oceánica y Atmosférica (NOAA) y, como puede ver en la URL de la API, este recurso se creó cuando el NCEI se llamaba NCDC. Si la URL de este recurso cambiara en el futuro, puede buscar "NCEI weather API" para encontrar la actualizada.*

## Configuración
Necesitamos importar `pandas` y leer los datos de formato largo para empezar:

In [ ]:
import pandas as pd
from datetime import datetime

long_df = pd.read_csv(
    'data/long_data.csv', usecols=['date', 'datatype', 'value']
).rename(
    columns={'value': 'temp_C'}
).assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_F=lambda x: (x.temp_C * 9/5) + 32
)
long_df.head()

## Transponer
La transposición intercambia las filas y las columnas. Para ello utilizamos el atributo `T`:

In [ ]:
long_df.set_index('date').head(6).T

## Pivotar
Pasar de formato largo a formato ancho.

### `pivot()`
Podemos reestructurar nuestros datos eligiendo una columna que irá en el índice (`index`), una columna cuyos valores únicos se convertirán en nombres de columna (`columns`), y los valores a colocar en esas columnas (`values`). El método `pivot()` se puede utilizar cuando no necesitamos realizar ninguna agregación además de nuestra reestructuración (cuando nuestro índice es único); si este no es el caso, necesitamos el método `pivot_table()` que veremos en el capítulo 4.

In [ ]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values='temp_C'
)
pivoted_df.head()

Ahora que los datos están pivotados, tenemos datos de formato ancho con los que podemos obtener estadísticas de resumen:

In [ ]:
pivoted_df.describe()

También podemos proporcionar múltiples valores sobre los que pivotar, lo que dará lugar a un índice jerárquico:

In [ ]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values=['temp_C', 'temp_F']
)
pivoted_df.head()

Con el índice jerárquico, si queremos seleccionar `TMIN` en Fahrenheit, tendremos que seleccionar primero `temp_F` y después `TMIN`:

In [ ]:
pivoted_df['temp_F']['TMIN'].head()

### `unstack()`

Hemos estado trabajando con un único índice a lo largo de este capítulo; sin embargo, podemos crear un índice a partir de cualquier número de columnas con `set_index()`. Esto nos da un índice de tipo `MultiIndex`, donde el nivel más externo corresponde al primer elemento de la lista proporcionada a `set_index()`:

In [ ]:
multi_index_df = long_df.set_index(['date', 'datatype'])
multi_index_df.head().index

Observe que ahora hay 2 secciones de índice en el marco de datos:

In [ ]:
multi_index_df.head()

Con un índice de tipo `MultiIndex`, ya no podemos utilizar `pivot()`. Ahora debemos utilizar `unstack()`, que por defecto mueve el índice más interno a las columnas:

In [ ]:
unstacked_df = multi_index_df.unstack()
unstacked_df.head()

El método `unstack()` también proporciona el parámetro `fill_value`, que nos permite rellenar cualquier valor `NaN` que pueda surgir de esta reestructuración de los datos. Consideremos el caso de que tengamos datos para la temperatura media del 1 de octubre de 2018, pero ninguna otra fecha:

In [ ]:
data = pd.DataFrame([{
    'datatype': 'TAVG', 
    'date': datetime(2018, 10, 1), 
    'temp_C': 10.0, 
    'temp_F': 50.0
}])
extra_data = pd.concat([long_df,data]).set_index(['date', 'datatype']).sort_index()

extra_data.loc['2018-10-01':'2018-10-02']

Si utilizamos `unstack()` en este caso, tendremos `NaN` para las columnas `TAVG` todos los días menos el 1 de octubre de 2018:

In [ ]:
extra_data.unstack().head()

Para solucionarlo, podemos introducir un valor de relleno apropiado. Sin embargo, estamos restringidos a pasar un valor para esto, no una estrategia (como vimos con `fillna()`), así que mientras `-40` definitivamente no es el mejor valor, podemos usarlo para ilustrar cómo funciona esto, ya que esta es la temperatura a la que Fahrenheit y Celsius son iguales:

In [ ]:
extra_data.unstack(fill_value=-40).head()

## Melting
Pasar de formato ancho a formato largo.

### Setup

In [ ]:
wide_df = pd.read_csv('data/wide_data.csv')
wide_df.head()

### `melt()`
Para pasar de formato ancho a formato largo, utilizamos el método `melt()`. Tenemos que especificar
- `id_vars`: qué columna(s) identifica(n) de forma única una fila en el formato ancho (`date`, aquí)
- `value_vars`: la(s) columna(s) que contiene(n) los valores (`TMAX`, `TMIN`, y `TOBS`, aquí)

Opcionalmente, también podemos proporcionar
- `value_name`: cómo llamar a la columna que contendrá todos los valores una vez fundidos
- `var_name`: cómo llamar a la columna que contendrá los nombres de las variables que se miden

In [ ]:
melted_df = wide_df.melt(
    id_vars='date',
    value_vars=['TMAX', 'TMIN', 'TOBS'],
    value_name='temp_C',
    var_name='measurement'
)
melted_df.head()

### `stack()`
Otra opción es `stack()`, que pivotará las columnas del marco de datos en el nivel más interno del índice (resultando en un índice de tipo `MultiIndex`). Para ilustrar esto, establezcamos nuestro índice en la columna `date`:

In [ ]:
wide_df.set_index('date', inplace=True)
wide_df.head()

Ejecutando ahora `stack()`, crearemos un segundo nivel en nuestro índice que contendrá los nombres de las columnas de nuestro dataframe (`TMAX`, `TMIN`, `TOBS`). Esto nos dejará con un objeto `Series` que contiene los valores:

In [ ]:
stacked_series = wide_df.stack()
stacked_series.head()

Podemos utilizar el método `to_frame()` en nuestro objeto `Series` para convertirlo en un objeto `DataFrame`. Como la serie no tiene nombre por el momento, le pasaremos el nombre como argumento:

In [ ]:
stacked_df = stacked_series.to_frame('values')
stacked_df.head()

Una vez más, tenemos un índice de tipo `MultiIndex`:

In [ ]:
stacked_df.head().index

Desgraciadamente, no tenemos un nombre para el nivel `datatype`:

In [ ]:
stacked_df.index.names

Sin embargo, podemos utilizar `set_names()` para solucionar este problema:

In [ ]:
stacked_df.index.set_names(['date', 'datatype'], inplace=True)
stacked_df.index.names

<hr>
<div>
    <a href="./3-cleaning_data.ipynb">
        <button>&#8592; Notebook Anterior</button>
    </a>
    <a href="./5-tratamiento_de_datos.ipynb">
        <button style="float: right;"> Siguiente Notebook &#8594;</button>
    </a>
</div>
<hr>